# Set CPU/Memory resource requirements for Minio

In [ ]:
from ckan_cloud_operator import kubectl

old_minio_pods = [pod for pod in kubectl.get('pod')['items'] if pod['metadata']['labels'].get('app') == 'provider-storage-minio']
assert len(old_minio_pods) == 1
old_minio_containers = old_minio_pods[0]['spec']['containers']
assert len(old_minio_containers) == 1

In [ ]:
import yaml
from IPython.core.display import HTML

HTML('<h3>OLD RESOURECS</h3>\n{}'.format(yaml.dump(old_minio_containers[0]['resources'], default_flow_style=False)))

## Set new resources

In [ ]:
import json
from ckan_cloud_operator.config import manager as config_manager

config_manager.set(
    key='container-spec-overrides', 
    value=json.dumps({
        "resources": {
            "requests": {
                "cpu": "1", 
                "memory": "1Gi"
            },
            "limits": {
                "memory":"2Gi"
            }
        }
    }),
    configmap_name='ckan-cloud-provider-storage-minio'
)

## Apply deployment: Dry Run

In [ ]:
from ckan_cloud_operator.providers.storage.minio import manager as minio_manager

minio_manager.initialize(dry_run=True)

## Apply Deployment

In [ ]:
from ckan_cloud_operator.providers.storage.minio import manager as minio_manager

minio_manager.initialize()

## Check deployment progress

In [ ]:
import yaml
from ckan_cloud_operator import kubectl

minio_pods = [pod for pod in kubectl.get('pod')['items'] if pod['metadata']['labels'].get('app') == 'provider-storage-minio']
pod_names = [pod['metadata']['name'] for pod in minio_pods]
print(yaml.dump(pod_names, default_flow_style=False))

[print(kubectl.check_output(f'describe pod {pod_name}').decode()) for pod_name in pod_names]